In [1]:
import seaborn as sb
import os
import time

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
#import numpy as np
#import os
#from itertools import combinations
#import random
import osmnx as ox

In [2]:
names = ["Visano", "Asola","Montichiari","Brescia","Roma"]
places = [(45.317468,10.3694991), # Visano (BS)
        (45.221332657542824, 10.41757427434648), # Asola (MN)
        (45.41301145796791, 10.39169903519818), # Montichiari (BS)
        (45.536282844234854, 10.206297121183406), # Brescia (BS)
        (41.90343248728792, 12.496484448996995)] # Roma (RO)

## Download maps and save graph as .xml

In [3]:
for c in places:
    name = names[places.index(c)]
    print(name)
    graph = ox.graph_from_point(c,dist=3000,network_type="drive")
    # conversions
    graph = nx.convert_node_labels_to_integers(graph)
    graph = graph.to_undirected()
    ox.save_graphml(graph,filepath='data/'+name+'_graph.xml')
    
    #ox.plot_graph(graph, node_size=2, figsize=(18,18))
 
    print(graph)
    

Visano
MultiGraph with 297 nodes and 378 edges
Asola
MultiGraph with 480 nodes and 664 edges
Montichiari
MultiGraph with 1130 nodes and 1509 edges
Brescia
MultiGraph with 3925 nodes and 5464 edges
Roma
MultiGraph with 4729 nodes and 7277 edges


## Load graphs

In [4]:
for name in names:
    
    filename = 'data/' + name + '_graph.xml'
    g = ox.load_graphml(filepath=filename)
    print(name)
    print(g)
    
    #ox.plot_graph(g, node_size=2, figsize=(18,18))

Visano
MultiGraph with 297 nodes and 378 edges
Asola
MultiGraph with 480 nodes and 664 edges
Montichiari
MultiGraph with 1130 nodes and 1509 edges
Brescia
MultiGraph with 3925 nodes and 5464 edges
Roma
MultiGraph with 4729 nodes and 7277 edges


## Dijkstra

Definizione della funzione per la creazione della stringa nella formattazione corretta per la lettura da parte di Minizinc

In [5]:
def dataStringGenerator(df, nome_variabile):
    stringa = nome_variabile +" = [";
    ## sezione per la matrice delle distanze
    if df.shape[0] == df.shape[1]:
        for i in range(df.shape[0]):
            if i == 0:
                stringa = stringa + "| "
            else:
                stringa = stringa + "         | "
            for j in range(df.shape[1]):
                stringa = stringa + str(df.iat[i, j]) 
                if j != len(df)-1:
                    stringa = stringa + ", "
                else:
                    stringa = stringa +  os.linesep # aggiunta di \n a fine della riga 

        stringa = stringa + "         |];"
    # sezione per i semplici array di numeri
    else:
        for i in range(df.shape[0]):
            if i != df.shape[0]-1:
                stringa = stringa + str(df.iat[i,0]) + ", "
            else:
                stringa = stringa + str(df.iat[i,0])
        
        stringa = stringa + "];"
    
    return stringa

In [6]:
mDists = []

for name in names:
    
    print(name)
    
    filename = 'data/' + name + '_graph.xml'
    g = ox.load_graphml(filepath=filename)
    
    
    labels = nx.get_edge_attributes(g, 'length')
    dictDist = dict(nx.all_pairs_dijkstra_path_length(g,weight='length'))
    mDist = pd.DataFrame(dictDist)
    #mDists.append(mDist)
    mDistSorted = mDist.sort_index()

    # mDistSorted contains the distances in meter
    # mDistSorted/500 means that in one minute the deliver travel 500 meters (30km/h) on average
    string = dataStringGenerator(pd.DataFrame(mDistSorted/500,dtype=int), "mdist")
    
    filename2write = 'data/' + name + '_dijkstra.dzn'
    file = open(filename2write, 'w')
    file.write(string + os.linesep)
    file.close()

    print(mDist.shape[0])

Visano
297
Asola
480
Montichiari
1130
Brescia
3925
Roma
4729
